In [217]:
#functions

import math
import random
del acc

# makes a ticket
def ticket_generator(t, o, a):
    ticket={
        'time': t,
        'owner': o,
        'action': a,
    }
    return ticket;


# depending on parameter (spread or cure), generates tau
def nextTime(rateParameter, n):
    taus=[0]*n
    for i in range(0,n): #start at zero, stops just short of n
        taus[i]=-math.log(1.0 - random.random()) / rateParameter
    return taus #generates n-length list, starting at index 0


#writes the edges to a network of n ppl where everyone is connected to everyone else 
def network_writer(n):
    connect=[0]*n
    for i in range(0, n): 
        a=list(range(n)) #generating a list that goes from 0 to n-1
        a.remove(i) #individual removes themselves from the network
        connect[i]=a #corresponds to the list of people person i is linked to
    return connect; #returns a list of lists with all the connections in the network

#ONCE NETWORKS ARE NOT SMALL WORLD, WILL NEED TO PASS IN CONNECT TO FUNCTION TO SEE WHO CAN BE INFECTED BY THE OWNER 
#(ow) AND TO GENERATE THE CORRECT LENGTH FOR Ts
#processes spread/infection tickets, returning a new timeline (with added tickets) and an updated list of states
def spreadtickethandler(tic, timeline, spreadparam, cureparam, ppl_state, t, t_max): #HEREEEEE
    ow=tic['owner']
    targ=tic['action'] #this action should not be -1 bc this is passed to spread ticket handler
    if (ow==targ):
        print('ow=targ, {:d}={:d}' .format(ow,targ))
#1
    if(ppl_state[ow]!=1):
        print('Error -- individual {:d} is not infected, but was passed to spreadtickethandler!' .format(ow))
        #break #is there a printf / how do you raise an error?  #SOMEHOW RAISE AND ERROR AND ESC
    else:
#2
        t=tic['time']
        if(ppl_state[targ]==1):
            print('{:d} is already infected! at time {:f}' .format(targ,t)) #targ=targ #just to make python shut up NOT REAL CODE
        else:
            ppl_state[targ]=1
            num_ppl=len(ppl_state) #number of people
            ts=[t]*(num_ppl-1) #list of t repeated num_ppl-1 times
            nT=nextTime(spreadparam, num_ppl-1) #generates taus
            cure_t=t+nextTime(cureparam, 1)[0] #using [0] in order to add an integer to t
            #curetimearray.append(cure_t) #curetime array -- purpose??
            spread_ts=[sum(x) for x in zip(ts,nT)] #using zip 
            for a in connect[ow]:
                if(spread_ts[acc]<cure_t and spread_ts[acc]<t_max): #HEREEE
                    tinf=ticket_generator(spread_ts[acc], targ, a)
                    add_to_timeline(tinf, timeline)
                acc=acc+1

#3
        #infected_count=sum(ppl_state) #will have something that checks the infected count at certain time intervals
#4
        spread_t2=t+nextTime(spreadparam,1)[0]
        cure_t_ow=t_max+1 #just so we don't get an error is cure_t_ow isn't generated in the while loop bc it's too big
        i=timeholder
        curefound=0 
        #looking for the curetime of each individual
        while curefound==0 and i<(len(timeline)-1): 
            tim=timeline[i]
            if(tim['owner']==ow): 
                if(tim['action']==-1):
                    cure_t_ow=tim['time']
                    curefound=1 #stop when cure is found
            i=i+1 
        if(spread_t2<cure_t_ow and spread_t2<t_max): #HEEREEE
            spread2tic=ticket_generator(spread_t2, ow, targ)
            add_to_timeline(spread2tic, timeline)
        return [timeline, ppl_state];
            
        
    
#processes cure tickets, returning the new states of ppl        
def curetickethandler(tic, ppl_state):
    ow=tic['owner']
    ppl_state[ow]=0
    #infected_count=sum(ppl_state) #will have something that checks the infected count at certain time intervals
    return ppl_state

#adds new tickets to timeline
def add_to_timeline(tic, tim):
    if not tim:
        return [tic]
    ind=len(tim)-1
    end=0
    while (tic['time']<tim[ind]['time'] and end==0):
        ind=ind-1 
        if(ind<0):
            end=1
    ind=ind+1
    tim.insert(ind, tic)
    return tim

def new_ind(ind, ow):
    if(ind>ow):
        ind=ind-1
    return ind

In [218]:

#Test ticket generator

tic=ticket_generator(1,1,1)
timeline=[]
print(timeline)

timeline=add_to_timeline(tic,timeline)

tic1=ticket_generator(4,1,1)
timeline=add_to_timeline(tic1,timeline)
#print(timeline)

tic2=ticket_generator(5,1,1)
timeline=add_to_timeline(tic2,timeline)
#print(timeline)

tic3=ticket_generator(3.5,1,1)
timeline=add_to_timeline(tic3,timeline)
#print(timeline)

tic4=ticket_generator(7,1,1)
timeline=add_to_timeline(tic4,timeline)
print(timeline)



[]
[{'time': 1, 'owner': 1, 'action': 1}, {'time': 3.5, 'owner': 1, 'action': 1}, {'time': 4, 'owner': 1, 'action': 1}, {'time': 5, 'owner': 1, 'action': 1}, {'time': 7, 'owner': 1, 'action': 1}]


In [219]:
#main

del timeline #deletes timeline from previous run

sick_ind=5
spreadparam=.5
cureparam=.3
curetimearray=[];
timeline=[]
num_ppl=100
ppl_state=[0]*num_ppl 
ppl_state[sick_ind]=1 #make a record of everyone's current state, starting with 1 infected person
connect=network_writer(num_ppl) #make a network of connections btwn the ppl -- everyone is connected to everyone rn
tinfinit=nextTime(spreadparam, num_ppl-1) #generate the time of the first infection THIS IS NOW ALL THE TIMES -- one for each
tcureinit=nextTime(cureparam, 1)[0] #generate the time of sick_ind's cure
while(all(i < tcureinit for i in tinfinit)): #for the very rare case that no individuals get infected before individual 0 gets cured
    tcureinit=nextTime(cureparam,1) #ensure that that time occurs after patient 0 has infected someone -- just so our sim goes somewhere
smallinds=[] #isolate only those times that are smaller than cure time
acc=0
for i in connect[sick_ind]:        
    if tinfinit[acc]<tcureinit:
        smallinds.append(i)
    acc=acc+1    
timeline=[]#initialize the timeline before any tickets are added
for i in range (len(smallinds)):
    init_inf=ticket_generator(tinfinit[new_ind(smallinds[i], sick_ind)], sick_ind, smallinds[i])#generate a ticket for the first infection 
    timeline=add_to_timeline(init_inf, timeline) #put these 2 tickets on timeline
#print(timeline)
init_cure=ticket_generator(tcureinit, sick_ind, -1) # generate a ticket for the first cure
timeline=add_to_timeline(init_cure, timeline)
#print(timeline)
timeholder=0 #timeholder is the index where we are on the timeline (start at 0)
t=0 #we start at a time of 0 
t_max=5 #we want to go to 500 time units



In [220]:
#ok time for the while loop
while t<t_max:
    print(t)
    if timeholder>(len(timeline)-1):
        print('end')
        break
    nexttic=timeline[timeholder] #look at the ticket corresponding to where we are on the timeline
    if (nexttic['action']==-1): #if it's a cure ticket, use the corresponding handler
        ppl_state=curetickethandler(nexttic, ppl_state)
    else: #if it's a spread ticket, use the corresponding handler
        vec=spreadtickethandler(nexttic, timeline, spreadparam, cureparam, ppl_state, t, t_max) #HEEREEE
        timeline=vec[0]
        ppl_state=vec[1]
    t=nexttic['time'] #new time based on the ticket we just read

    
    timeholder=timeholder+1 #increment timeholder
    


0


UnboundLocalError: local variable 'acc' referenced before assignment

In [212]:

print(i) 
print(timeline[i])
print(timeline)

74
{'time': 0.024614233616913006, 'owner': 5, 'action': 74}
[{'time': 0.024614233616913006, 'owner': 5, 'action': 86}, {'time': 0.024614233616913006, 'owner': 5, 'action': 0}, {'time': 0.024614233616913006, 'owner': 5, 'action': 1}, {'time': 0.024614233616913006, 'owner': 5, 'action': 2}, {'time': 0.024614233616913006, 'owner': 5, 'action': 3}, {'time': 0.024614233616913006, 'owner': 5, 'action': 4}, {'time': 0.024614233616913006, 'owner': 5, 'action': 6}, {'time': 0.024614233616913006, 'owner': 5, 'action': 7}, {'time': 0.024614233616913006, 'owner': 5, 'action': 8}, {'time': 0.024614233616913006, 'owner': 5, 'action': 9}, {'time': 0.024614233616913006, 'owner': 5, 'action': 10}, {'time': 0.024614233616913006, 'owner': 5, 'action': 11}, {'time': 0.024614233616913006, 'owner': 5, 'action': 12}, {'time': 0.024614233616913006, 'owner': 5, 'action': 13}, {'time': 0.024614233616913006, 'owner': 5, 'action': 14}, {'time': 0.024614233616913006, 'owner': 5, 'action': 15}, {'time': 0.024614233

In [ ]:
#test 

num_ppl=5
tinfinit=[1, 3, 2, 4, 5]
tcureinit=3

smallinds=[] #ERROR -- ULTIMATE GOAL is to isolate only those times that are smaller than cure time
for i in range (num_ppl-1):
    if tinfinit[i]<tcureinit:
        smallinds.append(i)

print(smallinds) 


In [22]:
spreadparam=.1
cureparam=.05
curetimearray=[];
timeline=[]
num_ppl=100
ppl_state=[0]*num_ppl 
ppl_state[0]=1 #make a record of everyone's current state, starting with 1 infected person
connect=network_writer(num_ppl) #make a network of connections btwn the ppl -- everyone is connected to everyone rn

tinfinit=nextTime(spreadparam, num_ppl-1) #generate the time of the first infection THIS IS NOW ALL THE TIMES -- one for each
tcureinit=nextTime(cureparam, 1)[0] #generate the time of patient 0's cure
while(sum(tcureinit>tinfinit)==0):
    tcureinit=nextTime(cureparam) #ensure that that time occurs after patient 0 has infected someone -- just so our sim goes somewhere
tinfinit.sort()   
smallinds=tinfinit<tcureinit
tinfinit=tinfinit[smallinds.index(1)]

TypeError: '>' not supported between instances of 'int' and 'list'

In [21]:
print(all(i < tcureinit[0] for i in tinfinit))

False


In [19]:
l1=[30, 34, 56]

all(i >= 30 for i in l1)

True

In [20]:
print(tcureinit[0])
print(tinfinit)

[0]
[18.900137608539172, 2.7595800380249704, 0.10891789736790652, 17.368996656500048, 2.45294843694846, 6.704430433308595, 0.1748955419401317, 4.8646342241154334, 11.385189763283636, 9.786465635853672, 0.11865745866570593, 6.04017066517331, 0.13194557214631525, 6.156042352291014, 32.92858509084251, 15.94384666225129, 6.194619658550465, 22.543115922067418, 3.4500118507255255, 6.655992288418583, 4.856863844482613, 8.595885931909567, 4.979684889996908, 0.646446599445525, 29.43224445115833, 12.046640268366074, 1.8582530884557515, 7.591699611724711, 11.030713762950151, 7.864876036411042, 12.724499345639916, 41.24963651337464, 18.183572075994398, 17.43939464120913, 1.9097157689220872, 15.819034496017071, 0.9813997084787515, 19.343086961366332, 7.619708482723527, 27.088677683866933, 9.386626724158903, 7.645786855686298, 15.308159662774958, 7.118874520953093, 7.754283793560688, 17.492311345666334, 13.301132055967459, 13.763362582429172, 1.2545543857978763, 21.75418121746059, 8.899197436873587,

In [ ]:
#old spreadtickethandler, give some ow=targ errors
def spreadtickethandler(tic, timeline, spreadparam, cureparam, ppl_state, t, t_max): #HEREEEEE
    ow=tic['owner']
    targ=tic['action'] #this action should not be -1 bc this is passed to spread ticket handler
    if (ow==targ):
        print('ow=targ, {:d}={:d}' .format(ow,targ))
#1
    if(ppl_state[ow]!=1):
        print('Error -- individual {:d} is not infected, but was passed to spreadtickethandler!' .format(ow))
        #break #is there a printf / how do you raise an error?  #SOMEHOW RAISE AND ERROR AND ESC
    else:
#2
        t=tic['time']
        if(ppl_state[targ]==1):
            print('{:d} is already infected! at time {:f}' .format(targ,t)) #targ=targ #just to make python shut up NOT REAL CODE
        else:
            ppl_state[targ]=1
            num_ppl=len(ppl_state) #number of people
            ts=[t]*(num_ppl-1) #list of t repeated num_ppl-1 times
            nT=nextTime(spreadparam, num_ppl-1) #generates taus
            cure_t=t+nextTime(cureparam, 1)[0] #using [0] in order to add an integer to t
            #curetimearray.append(cure_t) #curetime array -- purpose??
            spread_ts=[sum(x) for x in zip(ts,nT)] #using zip 
            acc=0
            for a in connect[ow]:
                if(spread_ts[acc]<cure_t and spread_ts[acc]<t_max): #HEREEE
                    tinf=ticket_generator(spread_ts[acc], targ, a)
                    add_to_timeline(tinf, timeline)
                acc=acc+1
            if cure_t<t_max: #HERE (unindent 2 lines below)
                tcure=ticket_generator(cure_t, targ, -1) #cure ticket
                add_to_timeline(tcure, timeline)
#3
        #infected_count=sum(ppl_state) #will have something that checks the infected count at certain time intervals
#4
        spread_t2=t+nextTime(spreadparam,1)[0]
        cure_t_ow=t_max+1 #just so we don't get an error is cure_t_ow isn't generated in the while loop bc it's too big
        i=timeholder
        curefound=0 
        #looking for the curetime of each individual
        while curefound==0 and i<(len(timeline)-1): 
            tim=timeline[i]
            if(tim['owner']==ow): 
                if(tim['action']==-1):
                    cure_t_ow=tim['time']
                    curefound=1 #stop when cure is found
            i=i+1 
        if(spread_t2<cure_t_ow and spread_t2<t_max): #HEEREEE
            spread2tic=ticket_generator(spread_t2, ow, targ)
            add_to_timeline(spread2tic, timeline)
        return [timeline, ppl_state];

In [ ]:
            smallinds=[] #isolate only those times that are smaller than cure time
            acc=0
            for i in connect[ow]:        
                if ts[acc]<cure_t:
                    smallinds.append(i)
                acc=acc+1    
            for i in range (len(smallinds)):
                tinf=ticket_generator(ts[new_ind(smallinds[i], ow)], ow, smallinds[i])#generate a ticket for the first infection 
                timeline=add_to_timeline(tinf, timeline) #put these 2 tickets on timeline                 
                
            if cure_t<t_max: #HERE (unindent 2 lines below)
                tcure=ticket_generator(cure_t, targ, -1) #cure ticket
                add_to_timeline(tcure, timeline)